## Sandbox

Un  `.ipynb` pour tester des idées :)

In [ ]:
using Pkg
Pkg.activate(".")
Pkg.instantiate()
using Graphs
using MetaGraphsNext
using JuMP
using Gurobi

ENV["GRB_LICENSE_FILE"] = "gurobi.lic"
include(joinpath("src", "utils.jl"));

In [ ]:
M = Model(Gurobi.Optimizer)

In [ ]:
infos = open(
    readInstance,
    joinpath("data", "20_USA-road-d.BAY.gr"),
);

In [ ]:
foreach(((key, value),) -> println("$key = $value"), pairs(infos))

In [ ]:
graph = graphFromData(infos);

In [ ]:
foreach(i -> println(graph[i]), labels(graph))

In [ ]:
foreach(((i, j),) -> println(graph[i, j]), edge_labels(graph))